# XQuery in Python: Process a whole collection at once on project XML files 

Source: <https://stackoverflow.com/questions/75142854/how-to-use-the-collection-function-with-saxonche> 

## New pip installs
* For this homework, you'll need to install the **pathlib** library so we can run XQuery across platforms on Mac *and* Windows. 
In your python environment for this series of notebooks, run `pip install pathlib`
* If you didn't install spacy before, be sure to `pip install spacy`
* Also, if you didn't download the spacy large language model, uncomment the line to download the spacy large model to your virtual environment.



In [26]:
# !pip install saxonche
# !pip install pathlib
import os
import pathlib
from pathlib import Path
import spacy
import re as regex
# re lets us work with regular expressions in Python
from saxonche import PySaxonProcessor
from os import getcwd
# this lets us retrieve the current working directory

Remember the spaCy language models? Let's try loading loading the large one to get the maximum amount of information from it! 
There's a lot we can experiment with from spaCy, so here's a link to the documentation for our ready reference:
<https://spacy.io/usage/spacy-101> 

We're going to start by just reviewing its POS (part of speech) and NER (named entity recognition) taggers to see what we can see in your project files.


In [27]:
#nlp = spacy.cli.download("en_core_web_lg")
# ONLY NEED ABOVE LINE ONCE. REMEMBER: COMMENT OUT THE ABOVE LINE THE NEXT TIME YOU RUN THIS.
nlp = spacy.load('en_core_web_lg')

Okay, let's explore some project files!
We've loaded the XML directory prepared by the Futurama team for our example here. 

* If you have some basic XML right now, like the Futurama team has prepared, we can easily scope in tagged sections of your collection. Swap out the Futurama collection with yours, and adjust the Python code below accordingly.
* If you don't have XML at this point, you can work around this over text files, or just explore the Futurama collection.

The next cell demonstrates the xpath() function, set up to run over individual files.
Let's look at how it returns information about distinct values of speakers. We're exploring distinct-values() and count() functions here. Try removing them and putting them back to see what the effect of distinct-values() is on the count. 


In [28]:
InputPath = 'XML-files'
OutputPath = 'testOutput' 

In [29]:
 def readTextFiles(InputPath):
    # This function uses XPath to read the XML input
    for file in os.listdir(InputPath):
        if file.endswith('.xml'):
            filepath = f"{InputPath}/{s21_e1_oscars-guy.xml}"
            with PySaxonProcessor(license=False) as proc:
                xml = open(filepath, encoding='utf-8').read()
                # ebb: Here we apply the Saxon processor to read files with XPath.
                xp = proc.new_xpath_processor()
                node = proc.parse_xml(xml_text=xml)
                xp.set_context(xdm_item=node)

                # From here on, we select the string that Python will send to NLP. 
                # xpath = xp.evaluate('//your/xpath/here')
                xpath = xp.evaluate('//character/@main => distinct-values() => sort()')
                count = xp.evaluate('//character/@main => distinct-values() => count()')
                string = str(xpath)
                print(xpath)
                # xpath is going to go file by file.
             
readTextFiles(InputPath)

FileNotFoundError: [Errno 2] No such file or directory: 'XML-files'

## Introducing XQuery! 
XQuery is what we want to use to help read data from across a whole directory, or "corpus" collection of files. 
XQuery can be written as a separate file (with .xql or .xquery extension) in oXygen over a directory. But we will find it more useful 
to apply it in Python if we're working on natural language processing applications. 

### Setting up XQuery in Python 
We use the same "boilerplate" PySaxonProcessor lines, but switch from xpath to the xquery processor.


Requirements: 
* We need all the xq lines to plug this processing into Python. You can use this code as a starter for your projects.
* The XQuery script is written inside a quoted block in the set_query_content() function that needs to take a quoted string, just like in the cell below.
* We need the run_query_to_value() function to execuit the script we're writing.
  

**Reading the collection**: We're setting this up to read a `collection()` function, which is a directory of XML files. 
**Writing XQuery**: This involves setting simple variables equal to xpath expressions. XQuery variables are defined with a `$`.
**XQuery Comments**: look like sideways smiley faces. `(: I'm an XQuery comment :)`

Let's take a look:



In [4]:
def xqueryOverFiles(InputPath):
    # This time, let's try XQuery over a collection of files:
    with PySaxonProcessor(license=False) as proc:
        print(proc.version)
        xq = proc.new_xquery_processor()
        # This only works on Mac / Linux: xq.set_query_base_uri('file://'+getcwd()+'/')
        xq.set_query_base_uri(Path('.').absolute().as_uri() + '/')
        xq.set_query_content('''
let $name := collection('fs21_e1_oscars-guy/?select=*.xml')
let $main := $name//character/@main => distinct-values() => sort()
let $count := count($main)
return $main

''')
        r = xq.run_query_to_value()
        print(r)  
                               
xqueryOverFiles(InputPath)

NameError: name 'InputPath' is not defined

## What's the difference? 

Notice that you return just one count for the entire collection, not 72 different counts for the speakers in each file.
How can we use this? 

We can get pull information from across the entire collection and find out literally who has the most speeches in the whole series. 
Take a look at this code. Don't worry if you don't know how to write it yet. I just want to show you for demonstration purposes! 
What's here is a nearly full "FLWOR statement" which stands for :

* For
* Let
* Where
* Order by
* Return

The only things required in a FLWOR statements are L and R. The others give you extra powers like we're seeing here.

**IMPORTANT: You're only allowed one return per FLWOR.**



In [ ]:
def xqueryOverFiles(InputPath):
    # This time, let's try XQuery over a collection of files:
    with PySaxonProcessor(license=False) as proc:
        print(proc.version)
        xq = proc.new_xquery_processor()
        xq.set_query_base_uri(Path('.').absolute().as_uri() + '/')
        xq.set_query_content('''
let $name := collection('XML-files/?select=*.xml')
let $main := $name//character/@main => distinct-values() => sort()
let $count := count($main)
for $na in $name
    let $count := $name//character[@who = $na] => count()
    where $count > 100
    order by $count descending
    return ($na || ':  ' || $count)
 
''')
        r = xq.run_query_to_value()
        print(r)  
                            

xqueryOverFiles(InputPath)


For this notice how we can move **deliberately** in XQuery from information on the whole collection, to information based on individuals in a series.
The counts we're seeing are NOT based on files, but on info about each speaker! 

## For statements, measurements, order by, return


* Let's choose a character from Futurama who gets a LOT of speeches and use XQuery to pull all their speeches from across the entire collection into one return.
The next code block is written to show you how to get all the speeches of one character in the WHOLE collection. You can change this to any other character you wish.
* Let's go through each speech individually and meaasure it: 
* Try out defining a new variable in the XQuery, to use the `string-length()` function, so you can measure the text of the speeches. Try it out!

* We're going to write a for statement so we can evaluate each speech, and order our results based on how long the speeches are,
     * We'll use the XPath `string-length()` function to measure each speech
     * We'll work through an XQuery `for` and `order by`
     * Let's see if we can return the speeches from shortest to longest, then try adding the word `descending` to order them longest to shortest!


* YOUR TURN: Edit the code below to return some alternative information! 

In [ ]:
def xqueryAndNLP(InputPath):
    # This time, let's try XQuery over a collection of files:
    with PySaxonProcessor(license=False) as proc:
        print(proc.version)
        xq = proc.new_xquery_processor()
        xq.set_query_base_uri(Path('.').absolute().as_uri() + '/')
        xq.set_query_content('''
let $futurama := collection('futurama-xml/?select=*.xml')
let $benderSpeeches := $futurama//speak[@who="BENDER"]
let $benderTextsOnly := $benderSpeeches/text() 
let $benders := for $bt in $benderTextsOnly
   let $length := $bt ! string-length()
   where $length gt 20
   order by $length descending
   return $bt
return string-join($benders, '\n\n')

''')
        # We started with this: r = xq.run_query_to_value() 
        # If you use this you need to convert r to a string in Python. r = str(run_query_to_value())
        # There's a run_query_to_string() option that's convenient so I'm using it here! 
        r = xq.run_query_to_string()
        print(r)

xqueryAndNLP(InputPath)

## Experimenting with some functions


Remember how we were going to apply some NLP from spaCy? The next part of this exercise is to take the output of this XQuery function and pass it to the spaCY language model!

We need XQuery in our function above to return just one string if we want to deliver that to spaCY and NLP tools. The way I left this code, it's returning thousands of strings.

* So here's your challenge: Write some code that returns all the text of a different speaker. (Or adjust the code to use in your project.)
* The `r` variable is storing the XQuery output. I have updated `r` to be xq.run_query_to_string() which should output a single string. If it is not, you can convert it `r` to single string in Python with `str()`.
* Then make sure it's returning a single string when you print(r). You may need to convert `r` 
* **Write some new code** that delivers this single string to be processed with spaCy in some way.
    * You can build your code in a new cell block below this one.
    *  You'll probably want to review [this spaCy NLP assignment](https://github.com/newtfire/textAnalysis-Hub/blob/main/python-nlp-exercise1.md#write-some-python-code-to-do-the-following)
    * Write your code to return any information of interest from spaCy, following the spaCy documentation as we did in that earlier assignment! Try looking for one or two different kinds of language. What are the most popular verbs, adjectives, nouns? Try out the NER (named entity recognition)...Write your code in your copy of this Jupyter Notebook to complete this exercise. 

In [33]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")
doc = nlp("This is a sentence.")
displacy.serve(doc, style="dep")

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.